# Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present

Production date: YYYY-MM-DD

**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Produced by: Olivier Burggraaff (National Physical Laboratory).

## 🌍 Use case: Use case listed here in full 

## ❓ Quality assessment question
* **In most cases there should be one question listed here in bold**

**‘Context paragraph’ (no title/heading)** - a very short introduction before the assessment statement describing approach taken to answer the user question. One or two key references could be useful, if the assessment summarises literature.

**Background**

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Finding 1
* Finding 2
* Finding 3
* etc
```

## 📋 Methodology

* Internal consistency: TempestExtremes vs TRACK/Hodges? (read papers)
* 

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.

[Windstorm tracks and footprints derived from reanalysis over Europe between 1940 to present
](https://doi.org/10.24381/bf1f06a9)

Hodges (TRACK) algorithm [[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2), [Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2), [Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)]

TempestExtremes [[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021), [Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)]

**Note:** This notebook is currently just a brain-dump in anticipation of starting the actual quality assessment at a later stage.

E.g. 'The analysis and results are organised in the following steps, which are detailed in the sections below:' 

**[](section-setup)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-analysis)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-results)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Code setup

#### Imports

In [ ]:
# Input / Output
from pathlib import Path
import earthkit.data as ekd
import warnings

# General data handling
import numpy as np
np.seterr(divide="ignore")  # Ignore divide-by-zero warnings
import pandas as pd
import geopandas as gpd
import xarray as xr
from functools import partial
from dask.array.core import PerformanceWarning
warnings.simplefilter(action="ignore", category=PerformanceWarning)

# Visualisation
import earthkit.plots as ekp
from earthkit.plots.styles import Style
import matplotlib.pyplot as plt
plt.rcParams["grid.linestyle"] = "--"
from matplotlib.colors import LogNorm
from cartopy import crs as ccrs
import cmcrameri as cmc
from tqdm import tqdm  # Progress bars

# Visualisation in Jupyter book -- automatically ignored otherwise
try:
    from myst_nb import glue
except ImportError:
    glue = None

#### Helper functions

##### General

In [ ]:
# Type hints for helper functions
from typing import Callable, Optional, Iterable

# For pre-defining functions
from functools import partial

##### Downloading data

In [ ]:
## Data downloading
def domain_to_request(domain: ekp.geo.domains.Domain) -> dict:
    """ From an earthkit-plots domain, generate a request for earthkit-data / cdsapi. """
    bbox = domain.bbox.to_latlon_bbox()

    # Round
    north = int(np.ceil(bbox.north) + 1)
    south = int(np.floor(bbox.south) - 1)
    west = int(np.floor(bbox.west) - 1)
    east = int(np.ceil(bbox.east) + 1)
    
    area = [north, west, south, east]
    return {"area": area}

##### Data (pre-)processing

In [ ]:
## Loops for convenience
def loop_over_(*args, progress=True, **kwargs) -> tqdm:
    """ Generate a tqdm progressbar; inverts `disable` keyword """
    return tqdm(*args, disable=not progress, leave=False, **kwargs)

def loop_over_ensemble_members(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over ensemble members in `data`, with a progress bar. """
    return loop_over_(data.groupby("number"), unit="member", **tqdm_kwargs)

def loop_over_data_variables(data: xr.Dataset, **tqdm_kwargs) -> tqdm:
    """ Loop over variable keys in `data`, with a progress bar. """
    return loop_over_(data.data_vars.keys(), unit="variable", **tqdm_kwargs)

##### Visualisation

In [ ]:
_style_chl = {"cmap": cmc.cm.navia.resampled(15), "norm": LogNorm(vmin=1e-6, vmax=50), "extend": "both"}

styles = {
    "chl": Style(**_style_chl),
}

# Apply general settings
for style in styles.values():
    style.normalize = False

In [ ]:
def _add_textbox_to_subplots(text: str, *axs: Iterable[plt.Axes | ekp.Subplot], right=False) -> None:
    """ Add a text box to each of the specified subplots. """
    # Get the plt.Axes for each ekp.Subplot
    axs = [subplot.ax if isinstance(subplot, ekp.Subplot) else subplot for subplot in axs]

    # Set up location
    x = 0.95 if right else 0.05
    horizontalalignment = "right" if right else "left"

    # Add the text
    for ax in axs:
        ax.text(x, 0.95, text, transform=ax.transAxes,
        horizontalalignment=horizontalalignment, verticalalignment="top",
        bbox={"facecolor": "white", "edgecolor": "black", "boxstyle": "round",
              "alpha": 1})

In [ ]:
def decorate_fig(fig: ekp.Figure, *, title: Optional[str]="") -> None:
    """ Decorate an earthkit figure with land, coastlines, etc. """
    # Add progress bar because individual steps can be very slow for large plots
    with tqdm(total=4, desc="Decorating", leave=False) as progressbar:
        fig.land()
        progressbar.update()
        fig.coastlines()
        progressbar.update()
        # fig.borders()
        # progressbar.update()
        fig.gridlines(linestyle=plt.rcParams["grid.linestyle"])
        progressbar.update()
        fig.title(title)
        progressbar.update()

(section-analysis)=
### 2. Analysis

[sis-european-wind-storm-reanalysis](https://doi.org/10.24381/bf1f06a9)

In [ ]:
dataset = "sis-european-wind-storm-reanalysis"

In [ ]:
# domain = ekp.geo.domains.Domain.from_string("Europe")
domain = "North Atlantic"
year = 2040

In [ ]:
request_default = {
    "variable": "all",
    "month": [f"{month:02}" for month in range(1, 13)],
    "day": [f"{day:02}" for day in range(1, 32)],
}

#### Windstorm summary indicator

In [ ]:
request = {
    "product": "windstorm_summary_indicator",
    "variable": "all",
    "tracking_algorithm": ["hodges"]
}

In [ ]:
data = ekd.from_source("cds", dataset, request)
data = data.to_pandas()
data

In [ ]:
# Set up GISCO URL and request
base = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/"
fname = "NUTS_RG_20M_2024_4326_LEVL_{level}.geojson"
urls = [base + "/" + fname.format(level=level) for level in (0, 1, 2)]

# Download data from GISCO
nuts_gdf = [gpd.read_file(url) for url in urls]

# Merge
nuts_gdf = gpd.GeoDataFrame(pd.concat(nuts_gdf))
nuts_gdf = nuts_gdf.rename(columns={"NUTS_ID": "region"})

In [ ]:
# Merge NUTS polygons into data
data_nuts = nuts_gdf.merge(data, on="region")

# Filter only desired columns
data_nuts = data_nuts[["region", "year", "threshold", "algorithm",  # Selection
                       "storms_number", "mean_wind_gust", "ssi", "normalised_ssi", "area_ratio", "wind_gust_ratio",  # Variables
                       "LEVL_CODE", "CNTR_CODE", "NAME_LATN", "geometry",  # NUTS
                     ]]

In [ ]:
selection = (data_nuts["year"] == 2004) & \
            (data_nuts["threshold"] < 1) & \
            (data_nuts["algorithm"] == "hodges") & \
            (data_nuts["LEVL_CODE"] == 0)

data_national = data_nuts[selection]
data_national = data_national.to_crs("EPSG:3035")

data_national.plot.geo("storms_number")
plt.show()
plt.close()

In [ ]:
# For earthkit-plots issue
# data_nuts[selection].to_file("choropleth_example.geojson", driver='GeoJSON')

In [ ]:
# # Show variables
# # data_plot = data.sel(track_id=track_id)

# fig = ekp.Figure(rows=1, columns=1, size=(4, 4))

# # Plot single variable
# subplot = fig.add_map(domain=domain)
# # data_nuts[selection].plot.geo("storms_number", ax=subplot.ax, transform=subplot.crs)
# subplot.shapes(data_nuts[selection])
# # subplot.grid_cells(data_plot, z="footprint", style=style)

# # # Legend at the bottom
# # for subplot in fig.subplots:
# #     try:
# #         subplot.legend(location="bottom")
# #     except ValueError:
# #         continue

# # Decorate figures
# # decorate_fig(fig)

# # Show result
# plt.show()
# plt.close()

#### Windstorm footprint

In [ ]:
request = {
    "product": "windstorm_footprint",
    "tracking_algorithm": ["hodges"],
    "event_aggregation": "single_event",
    "windstorm_footprint_resolution": "original",
    "spatial_extent": "full_domain",
    "year": ["2003", "2004", "2005"],
} | request_default

In [ ]:
data = ekd.from_source("cds", dataset, request)
data = data.to_xarray()
data

In [ ]:
# Show variables
track_id = 1366
data_plot = data.sel(track_id=track_id)

fig = ekp.Figure(rows=1, columns=1, size=(4, 4))

# Plot single variable
style = Style(**{"cmap": plt.cm.cividis, "vmin": 0, "vmax": 40, "normalize": False})

subplot = fig.add_map(domain=domain)
subplot.grid_cells(data_plot, z="footprint", style=style)

# Legend at the bottom
for subplot in fig.subplots:
    try:
        subplot.legend(location="bottom")
    except ValueError:
        continue

# Decorate figures
decorate_fig(fig)

# Show result
plt.show()
plt.close()

#### Windstorm track

In [ ]:
request = {
    "product": "windstorm_track",
    "tracking_algorithm": ["hodges"],
    "event_aggregation": "single_event",
    "year": ["2003", "2004", "2005"],
} | request_default

In [ ]:
data = ekd.from_source("cds", dataset, request)
data = data.to_pandas()
data["time"] = pd.to_datetime(data["time"])
data

In [ ]:
# Show variables
track_id = 1366
data_plot = data[data["id"] == track_id].set_index("time")

fig = ekp.Figure(rows=1, columns=1, size=(4, 4))

# Plot single variable
style = Style(**{"cmap": plt.cm.magma, "vmin": 0, "vmax": 20, "normalize": False})

subplot = fig.add_map(domain=domain)
subplot.scatter(x=data_plot["longitude"], y=data_plot["latitude"], c=data_plot["fg10"],
                cmap=plt.cm.magma, vmin=0, vmax=20, zorder=2)
subplot.line(x=data_plot["longitude"], y=data_plot["latitude"], c="k", zorder=1)

# Legend at the bottom
# N.B. Currently doesn't work because it looks for a style in the subplot.line call
for subplot in fig.subplots:
    try:
        subplot.legend(location="bottom", style=style)
    except ValueError:
        continue

# Decorate figures
decorate_fig(fig)

# Show result
plt.show()
plt.close()

(section-results)=
### 5. Results

#### Results Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here. 

In [ ]:
# collapsable code cell

# code is included for transparency but also learning purposes and gives users the chance to adapt the code used for the assessment as they wish

## ℹ️ If you want to know more

### Key resources

List some key resources related to this assessment. E.g. CDS entries, applications, dataset documentation, external pages.
Also list any code libraries used (if applicable).

Code libraries used:
* [earthkit](https://github.com/ecmwf/earthkit)
  * [earthkit-data](https://github.com/ecmwf/earthkit-data)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

### References
[[Hoskins+02](https://doi.org/10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2)] B. J. Hoskins and K. I. Hodges, ‘New Perspectives on the Northern Hemisphere Winter Storm Tracks’, Journal of the Atmospheric Sciences, vol. 59, no. 6, pp. 1041–1061, Mar. 2002, doi: 10.1175/1520-0469(2002)059%3C1041:NPOTNH%3E2.0.CO;2.

[[Hodges+99](https://doi.org/10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2)] K. I. Hodges, ‘Adaptive Constraints for Feature Tracking’, Monthly Weather Review, vol. 127, no. 6, pp. 1362–1373, June 1999, doi: 10.1175/1520-0493(1999)127%3C1362:ACFFT%3E2.0.CO;2.

[[Hodges+95](https://doi.org/10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2)] K. I. Hodges, ‘Feature Tracking on the Unit Sphere’, Monthly Weather Review, vol. 123, no. 12, pp. 3458–3465, Dec. 1995, doi: 10.1175/1520-0493(1995)123%3C3458:FTOTUS%3E2.0.CO;2.

[[Ullrich+21](https://doi.org/10.5194/gmd-14-5023-2021)] P. A. Ullrich, C. M. Zarzycki, E. E. McClenny, M. C. Pinheiro, A. M. Stansfield, and K. A. Reed, ‘TempestExtremes v2.1: a community framework for feature detection, tracking, and analysis in large datasets’, Geoscientific Model Development, vol. 14, no. 8, pp. 5023–5048, Aug. 2021, doi: 10.5194/gmd-14-5023-2021.

[[Ullrich+17](https://doi.org/10.5194/gmd-10-1069-2017)] P. A. Ullrich and C. M. Zarzycki, ‘TempestExtremes: a framework for scale-insensitive pointwise feature tracking on unstructured grids’, Geoscientific Model Development, vol. 10, no. 3, pp. 1069–1090, Mar. 2017, doi: 10.5194/gmd-10-1069-2017.